In [29]:

from sounda_voice.text2speech import Text2Speech as Text2Speech_sounda
from text2speech import Text2Speech as Text2Speech_vits
import pypinyin
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # 多行都显示
from IPython.display import Audio
import utils_audio
import os



# Debug

## 播放text2speech在`DEBUG=True`模式时记录的最后一次语音合成

In [20]:
import pickle 
with open("./output/response.pkl", "rb") as frb:
    a = pickle.load(frb)
    a = json.loads(a)

import numpy as np
import json
import base64
from IPython.display import Audio
#base64.b64decode

Audio(np.frombuffer(base64.b64decode(a['audio_buffer']),dtype=np.int16), rate=int(a['sr']))


## 播放speech2text在`DEBUG=True`模式时每次清空buffer前记录的音频

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # 多行都显示
from IPython.display import Audio

!python speech2text.py debug_audio_1701020802487.wav medium /root/autodl-fs/whisper

Audio("debug_audio_1701021182440.wav")

/root/miniconda3/lib/python3.8/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
[2023-11-27 02:02:57-INFO]: >>> use FILE_FP as 'debug_audio_1701020802487.wav'
[2023-11-27 02:02:57-INFO]: >>> use MODEL_TYPE as 'medium'
[2023-11-27 02:03:08-INFO]: >>> try to transcribe with prehot_audio
[2023-11-27 02:03:12-INFO]: >>> transcribe_file:
我刚才又停了将近一秒钟
[2023-11-27 02:03:14-INFO]: >>> transcribe_file(lang=zh):
最近几日,我在考虑家宅布置的事。


## 分chunk加载音频

In [69]:
import wave
import os
from IPython.display import Audio
import numpy as np

fp = os.path.abspath("./prehot_speech2text.wav")
print("file: %s" % fp)
Audio(fp)
with wave.open(fp, 'rb') as wf:
    print("声道数: %s" % wf.getnchannels())
    print("采样率: %s" % wf.getframerate()) # 一秒多少个样本（样本数*样本宽就是一秒多少字节）
    print("采样总数(样本总数、数组长度): %s" % wf.getnframes())
    print("采样宽度(样本宽度): %s" % wf.getsampwidth())  # 标准的16位PCM音频中，每个样本(即int16)占用2个字节
    print("音频总时长: %s" % (wf.getnframes()/wf.getframerate()))
    
    standard_dtypes = {2:np.int16, 4:np.int32}
    dtype = standard_dtypes[wf.getsampwidth()]
    all_time,each_time = 0,1.0
    while all_time < wf.getnframes()/wf.getframerate():
        chunk_size = int(wf.getframerate() * each_time)  # 采样率*采样时间拿到样本数（取整）
        buffer = wf.readframes(chunk_size)
        # 注意这里的逻辑：
        # 每次循环读取each_time时长的样本，对应的是chunk_size个样本
        # 每次拿到的样本字节流长度（即buffer长度）是chunk_size*sample_width
        # 所以最后一次拿到的buffer想要补零的话：
        #  - 首先计算本次实际拿到的“样本个数”是 len(buffer)//sample_width
        #  - 然后用numpy补上 “chunk_size-样本个数” 的零
        print(">>> eatch_time:%s, chunk_size: %s, buffer_len: %s, buffer2arr_shape: %s" % (each_time,chunk_size,len(buffer),np.frombuffer(buffer,dtype=dtype).shape))
        if(len(buffer)//wf.getsampwidth() < chunk_size):
            pad_zeros = np.zeros((chunk_size-len(buffer)//wf.getsampwidth()),dtype=dtype)
            pad_res = np.hstack([np.frombuffer(buffer,dtype=dtype), pad_zeros])
            buffer = pad_res.tobytes()
            print("after padding a shape=%s zeros, buffer_len: %s" % (pad_zeros.shape, len(buffer)))
        Audio(np.frombuffer(buffer, dtype=dtype), rate=wf.getframerate())
        all_time += each_time

file: /root/Serving_VITS-fast-fine-tuning/prehot_speech2text.wav


声道数: 1
采样率: 22050
采样总数(样本总数、数组长度): 71678
采样宽度(样本宽度): 2
音频总时长: 3.250702947845805
>>> eatch_time:1.0, chunk_size: 22050, buffer_len: 44100, buffer2arr_shape: (22050,)


>>> eatch_time:1.0, chunk_size: 22050, buffer_len: 44100, buffer2arr_shape: (22050,)


>>> eatch_time:1.0, chunk_size: 22050, buffer_len: 44100, buffer2arr_shape: (22050,)


>>> eatch_time:1.0, chunk_size: 22050, buffer_len: 11056, buffer2arr_shape: (5528,)
after padding a shape=(16522,) zeros, buffer_len: 44100


# Sounda

In [53]:
pypinyin.load_phrases_dict({"映射测试": [["ce4"], ["shi4"], ["ying4"], ["she4"]]}, style=pypinyin.Style.TONE3)

M = Text2Speech_sounda(encoder_fp="/root/models/pretrained1.pt",
                synth_fp="/root/models/model_zn_v2.pt",  # ~/models/pretrained-11-7-21_75k.pt
                vocoder_fp="/root/models/g_hifigan.pt")

# M = Text2Speech_sounda(encoder_fp="/root/models/pretrained1.pt",
#                 synth_fp="/root/models/pretrained-11-7-21_75k.pt",  # 
#                 vocoder_fp="/root/models/g_hifigan.pt")

M.init()

Loaded encoder "pretrained1.pt" trained to step 1594501
Building hifigan
Loading '/root/models/g_hifigan.pt'
Complete.
Removing weight norm...
